In this programming assignment, we're going to be implementing an MDP, Value Iteration and Policy Iteration, and at Belief State Updates for a POMDP.

# MDP formulation

The first step is to formulate the MDP using a Transition function, T, and a Reward function, R. Use the templates below and the parameters from problem 2 in the written component to implement T and R.

## Transition function

In [ ]:
def T(s, a, s_):
    """Transition function from state s to state s_ via action a."""
    
    if s == 'healthy':
        
        if a == 'vegetables':
            if s_ == 'healthy':
                return 1
            elif s_ == 'sick':
                return 0
            elif s_ == 'toothless':
                return 0
            else:
                print('Specify valid state for s_: healthy, sick, or toothless')
                return
            
        elif a == 'candy':
            if s_ == 'healthy':
                return 1/4
            elif s_ == 'sick':
                return 3/4
            elif s_ == 'toothless':
                return 0
            else:
                print('Specify valid state for s_: healthy, sick, or toothless')
                return
        else:
            print('Specify valid action for a: candy or vegetables')
            return
    
    elif s == 'sick':
        if a == 'vegetables':
            if s_ == 'healthy':
                return 1/4
            elif s_ == 'sick':
                return 3/4
            elif s_ == 'toothless':
                return 0
            else:
                print('Specify valid state for s_: healthy, sick, or toothless')
                return
        elif a == 'candy':
            if s_ == 'healthy':
                return 0
            elif s_ == 'sick':
                return 7/8
            elif s_ == 'toothless':
                return 1/8
            else:
                print('Specify valid state for s_: healthy, sick, or toothless')
                return
        else:
            print('Specify valid action for a: candy or vegetables')
            return

    elif s == 'toothless':
        if s_ != 'toothless':
            return 0
        elif s_ == 'toothless':
            return 1
    
    else:
        print('Specify valid state for s: healthy, sick, or toothless.')
        return 

## Reward function

In [ ]:
# STUDENT
def R(s, a, s_):
    if s == 'toothless':
        return 0
    elif a == 'candy':
        return 10
    elif a == 'vegetables':
        return 4
    else:
        print('Specify valid action for a: candy or vegetables')
        return

# MDP Solvers

Now that you've implemented T and R, implement Value Iteration and Policy Iteration using the templates below and the pseudocode presented in class or in the book.

## Value Iteration

In [ ]:
S = ["healthy", "sick", "toothless"]
A = ["candy", "vegetables"]
epsilon = 0.0001
gamma = 0.9

def valueIteration(S, A, T, R, gamma, epsilon):
    U = {'healthy': 0, 'sick': 0, 'toothless': 0}
    U_ = {'healthy': 0, 'sick': 0, 'toothless': 0}
    pi = {}
    delta = 1

    while gamma * delta > epsilon*(1-gamma):
        U = dict(U_)
        delta = 0
        for state in S:
            max_u = 0
            for action in A:
                u = R(state, action, '')
                for state_ in S:
                    u += gamma * T(state, action, state_) * U[state_]
                if u >= max_u:
                    max_u = u
                    pi[state] = action
            U_[state] = max_u
            if abs(U_[state] - U[state]) > delta:
                delta = abs(U_[state] - U[state]) 

    if gamma == 0:
        max_u = 0
        for action in A:   
            if R('', action, '') > max_u:
                max_u = R('', action, '')
                max_act = action
        for state in S:
            pi[state] = max_act
     
    return pi 
    # the policy as a dictionary 
    # e.g. pi = {'healthy': 'vegetables', 'sick': 'candy', 'toothless': 'vegetables'}

## Policy Iteration

In [ ]:
S = ["healthy", "sick", "toothless"]
A = ["candy", "vegetables"]
epsilon = 0.0001
gamma = 0.9

def policyIteration(S, A, T, R, gamma, epsilon):
    U = {'healthy': 0, 'sick': 0, 'toothless': 0}
    pi = {'healthy': 'candy', 'sick': 'candy', 'toothless': 'candy'}
    unchanged = False

    while not unchanged:
        for state in S:
            u = R(state, pi[state], '')
            for state_ in S:
                u += gamma * T(state, pi[state], state_) * U[state_]
            U[state] = u
        unchanged = True
        for state in S:
            max_util = U[state]
            for action in A:
                u = R(state, action, '')
                for state_ in S:
                    u += gamma * T(state, action, state_) * U[state_]
                if u > max_util:
                    max_util = u
                    pi[state] = action
                    unchanged = False
    return pi
    # the policy as a dictionary 
    # e.g. pi = {'healthy': 'vegetables', 'sick': 'candy', 'toothless': 'vegetables'}

## Finding the $\gamma$ threshold

You may have noticed that the optimal policy is typically to eat candy in every state. Find the smallest, up to 0.0001 error, discount factor $\gamma$ such that for $\pi$, the optimal policy, $\pi$(sick) = vegetables.

In [ ]:
def find_gamma(solver, epsilon=0.0001):
    gamma = 1
    pi = solver(S, A, T, R, 0.9, epsilon)
    while pi['sick'] == 'vegetables':
        gamma -= 0.1
        pi = solver(S, A, T, R, gamma, epsilon)
    while pi['sick'] != 'vegetables':
        gamma += 0.01
        pi = solver(S, A, T, R, gamma, epsilon)
    while pi['sick'] == 'vegetables':
        gamma -= 0.001
        pi = solver(S, A, T, R, gamma, epsilon)
    while pi['sick'] != 'vegetables':
        gamma += 0.0001
        pi = solver(S, A, T, R, gamma, epsilon)
    gamma_min = gamma    
    
    return gamma_min

# POMDP Belief States

In problem 3 of the written you computed belief state updates by hand. To do this you had to compute the evidence posterior $\mathbb P(e | s)$ and run the FORWARD updates $$b(s') = \alpha \mathbb P(e|s') \sum_{s \in S} \mathbb P(s' | s, a) b(s).$$
Implement the evidence posterior, O, below and the update_belief_state method using the FORWARD update. You can use this to check your answer for problem 3(c) in the written.

## Evidence posterior

In [ ]:
def O(evidence, state):
    """Returns probably of observing evidence ('F' if toothless, 0 if T < 100, 1 if T >= 100) given state."""

    if evidence == 'F':
        if state == 'toothless':
            return 1
        else:
            return 0
    else:
        temperature = evidence
        # State Posteriors
        P_healthy_given_T_minus = 0.9
        P_sick_given_T_minus = 0.1
        
        # Evidence Posteriors
        P_healthy_given_T_plus = 0.01 # FILL IN
        P_sick_given_T_plus = 0.99 # FILL IN
        
        # Evidence Priors
        P_T_minus = 0.9438 # FILL IN
        P_T_plus = 0.0562 # FILL IN
        
        # State Priors
        P_healthy = 0.85 # FILL IN
        P_sick = 0.15 # FILL IN

        if temperature == 0:
            
            P_temperature = P_T_minus
            
            if state == 'healthy':
                P_state_given_temperature = P_healthy_given_T_minus
                P_state = P_healthy
            elif state == 'sick':
                P_state_given_temperature = P_sick_given_T_minus
                P_state = P_sick
            elif state == 'toothless':
                P_state_given_temperature = 0
                P_state = 0.01
            else:
                print('Specify healthy or sick for state!')
                return
            
        elif temperature == 1:
            
            P_temperature = P_T_plus
            
            if state == 'healthy':
                P_state_given_temperature = P_healthy_given_T_plus
                P_state = P_healthy
            elif state == 'sick':
                P_state_given_temperature = P_sick_given_T_plus
                P_state = P_sick
            elif state == 'toothless':
                P_state_given_temperature = 0
                P_state = 0.01
            else:
                print('Specify healthy or sick for state!')
                return

        else:
            print('Specify binary variable for temperature!')
            return
        
        P_temperature_given_state = P_state_given_temperature * P_temperature / P_state # FILL IN
        
        return P_temperature_given_state # Update probability computed with Bayes' rule

## Belief state update 

In [ ]:
def update_belief_state(temperature, action, belief_state):
    s = ['healthy', 'sick', 'toothless']
    b_list = []
    for i in range(0, len(belief_state)):
        b_ = 0
        for j in range(0, len(belief_state)):
            b_ += T(s[j], action, s[i]) * belief_state[j]
        b_list.append(b_ * O(temperature, s[i]))
    b_0 = b_list[0] / sum(b_list)
    b_1 = b_list[1] / sum(b_list)
    b_2 = b_list[2] / sum(b_list)

    updated_belief_state = (b_0, b_1, b_2)
    return updated_belief_state 
    # 3-tuple (b_0', b_1', b_2') representing updated probabilities of each state {"healthy", "sick", "toothless"}